In [ ]:
!nvidia-smi

Tue Jun  1 05:49:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
from datetime import datetime
import csv
import os
import sys
import xml.etree.ElementTree as ET
import shutil
import random

In [ ]:
ALL_CLASSES = ["aquarium", "bottle", "bowl", "box", "bucket", "plastic_bag", "plate", "styrofoam", "tire", "toilet", "tub", "washing_machine", "water_tower"]
TRAIN_CLASSES = [
    'Toilet', 'Bathtub', 'Plastic bag'
]
CDC_TRAIN_SIZE = 1
BACKGROUND_IMG_SIZE = 1000
random.seed(531)

In [ ]:
cdc_openImages_mapping = {
    'toilet': 'Toilet',
    'tub': 'Bathtub',
    'plastic_bag': 'Plastic bag',
    'washing_machine': 'Washing machine'
}
openImages_cdc_mapping = {y: x for x, y in cdc_openImages_mapping.items()}

# Preparation

In [ ]:
!pip install -q boto3

     |████████████████████████████████| 133kB 24.6MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 7.6MB 41.9MB/s 
     |████████████████████████████████| 143kB 44.5MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [ ]:
!mkdir open_images_v6
%cd /content/open_images_v6
!wget -q https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv
!wget -q https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv
!wget -q https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
%cd /content
!wget -q https://raw.githubusercontent.com/openimages/dataset/master/downloader.py

/content/open_images_v6
/content


In [ ]:
with open('open_images_v6/class-descriptions-boxable.csv') as class_file:
  reader = csv.reader(class_file)
  class_to_label = {x[1]: x[0] for x in reader}
print("Total classes: {}".format(len(class_to_label)))

Total classes: 601


In [ ]:
!rm -rf open_images_v6/train
!rm -rf open_images_v6/validation
!mkdir -p open_images_v6/train/labels
!mkdir -p open_images_v6/validation/labels

def create_labels(dataset_type, box_path):
  image_set = set()
  all_image_set = set()
  bad_image_set = set()
  with open(box_path) as box_file:
    rows = csv.DictReader(box_file)
    for row in rows:
      all_image_set.add(row['ImageID'])
      if row['LabelName'] in labels:
        if row['IsGroupOf'] == '1' or row['IsDepiction'] == '1':
          bad_image_set.add(row['ImageID'])
          continue
        xmin = float(row['XMin'])
        xmax = float(row['XMax'])
        ymin = float(row['YMin'])
        ymax = float(row['YMax'])
        bbox = [
          (xmin + xmax)/2,
          (ymin + ymax)/2,
          xmax - xmin,
          ymax - ymin
        ]
        class_id = labels.index(row['LabelName'])
        s = str(class_id) + ' ' + ' '.join(['{:.6f}'.format(x) for x in bbox])
        label_file_path = 'open_images_v6/{}/labels/{}.txt'.format(dataset_type, row['ImageID'])
        with open(label_file_path, 'a') as label_file:
          if row['ImageID'] in image_set:
            label_file.write('\n')
          else:
            image_set.add(row['ImageID'])
          label_file.write(s)
  print(len(image_set), len(bad_image_set), len(all_image_set))
  image_set = image_set - bad_image_set
  if dataset_type == 'train':
    background_image_list = random.sample(all_image_set - bad_image_set - image_set, BACKGROUND_IMG_SIZE)
    image_set.update(background_image_list)
  print(len(image_set), len(bad_image_set), len(all_image_set))
  return image_set

labels = [class_to_label[x] for x in TRAIN_CLASSES]
image_train_set = create_labels('train', 'open_images_v6/oidv6-train-annotations-bbox.csv')
image_validation_set = create_labels('validation', 'open_images_v6/validation-annotations-bbox.csv')

1793 114 1743042
2755 114 1743042
47 6 37306
47 6 37306


In [ ]:
print(len(image_train_set), len(image_validation_set))
image_train_list = ['train/{}'.format(x) for x in image_train_set]
image_validation_list = ['validation/{}'.format(x) for x in image_validation_set]
with open('open_images_v6/train_list.txt', 'w') as file:
  file.write('\n'.join(image_train_list))
with open('open_images_v6/validation_list.txt', 'w') as file:
  file.write('\n'.join(image_validation_list))

2755 47


In [ ]:
!python downloader.py open_images_v6/train_list.txt \
                      --download_folder=open_images_v6/train/images
!python downloader.py open_images_v6/validation_list.txt \
                      --download_folder=open_images_v6/validation/images

In [ ]:
def resize(path):
  root, _, filenames = next(os.walk(path))
  for filename in filenames:
    image_path = os.path.join(root, filename)
    im = Image.open(image_path)
    im.thumbnail((640,640), Image.LANCZOS)
    im.save(image_path)

resize('open_images_v6/train/images')
resize('open_images_v6/validation/images')

In [ ]:
%cd /content
# train dataset
!cp drive/MyDrive/作業/深度學習概論與應用/cdc/train_cdc.zip .
!unzip -q train_cdc.zip
# test dataset
!cp drive/MyDrive/作業/深度學習概論與應用/cdc/test_cdc.zip .
!unzip -q test_cdc.zip

/content


In [ ]:
!rm -rf train_cdc/images
!rm -rf train_cdc/labels
!mkdir -p train_cdc/images/train
!mkdir -p train_cdc/images/val
!mkdir -p train_cdc/labels/train
!mkdir -p train_cdc/labels/val

def convert(size, box):
  dw = 1./size[0]
  dh = 1./size[1]
  x = (box[0] + box[1])/2.0
  y = (box[2] + box[3])/2.0
  w = box[1] - box[0]
  h = box[3] - box[2]
  x = x*dw
  w = w*dw
  y = y*dh
  h = h*dh
  return (x,y,w,h)

def convert_annotation(image_id):
  in_file = open('train_cdc/train_annotations/%s.xml'%(image_id))
  dataset_type = 'train' if random.random() <= CDC_TRAIN_SIZE else 'val'
  shutil.copy2('train_cdc/train_images/%s.jpg'%(image_id), 'train_cdc/images/%s'%(dataset_type))
  tree = ET.parse(in_file)
  root = tree.getroot()
  size = root.find('size')
  w = int(size.find('width').text)
  h = int(size.find('height').text)

  s = str()
  for obj in root.iter('object'):
    cls = obj.find('name').text
    if cls not in cdc_classes:
      continue
    cls_id = cdc_classes.index(cls)
    xmlbox = obj.find('bndbox')
    b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
    bb = convert((w,h), b)
    if (s != ''): s += '\n'
    s += str(cls_id) + ' ' + ' '.join(['{:.6}'.format(a) for a in bb])
  with open('train_cdc/labels/%s/%s.txt'%(dataset_type, image_id), 'w') as out_file:
    out_file.write(s)

_, _, filenames = next(os.walk('train_cdc/train_images'))
image_ids = [os.path.splitext(filename)[0] for filename in filenames]
cdc_classes = [openImages_cdc_mapping[x] for x in TRAIN_CLASSES]

for image_id in image_ids:
  convert_annotation(image_id)

# Train

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -q -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 6782, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 6782 (delta 49), reused 54 (delta 27), pack-reused 6696
Receiving objects: 100% (6782/6782), 8.74 MiB | 31.95 MiB/s, done.
Resolving deltas: 100% (4656/4656), done.
     |████████████████████████████████| 645kB 29.0MB/s 
     |████████████████████████████████| 133kB 54.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [ ]:
preset = '''
train: [/content/open_images_v6/train/images, /content/open_images_v6/validation/images, /content/train_cdc/images/train]
val: /content/train_cdc/images/train
nc: {nc}
names: {names}
'''
names = '["' + '", "'.join(TRAIN_CLASSES) + '"]'
with open('open_images_v6/dataset.yaml', 'w') as file:
  file.write(preset.format(nc=len(TRAIN_CLASSES),names=names))

In [ ]:
%cd /content
!cp yolov5/data/hyp.scratch.yaml open_images_v6/hyp.yaml

/content


!sed -i 's/degrees:.*/degrees: 10.0/' open_images_v6/hyp.yaml
!sed -i 's/translate:.*/translate: 0.1/' open_images_v6/hyp.yaml
!sed -i 's/scale:.*/scale: 0.1/' open_images_v6/hyp.yaml
!sed -i 's/shear:.*/shear: 10.0/' open_images_v6/hyp.yaml
!sed -i 's/mosaic:.*/mosaic: 0.0/' open_images_v6/hyp.yaml

In [ ]:
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 50 \
                --weights yolov5m6.pt \
                --data /content/open_images_v6/dataset.yaml

/content/yolov5
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-130-gfdbe527 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/open_images_v6/dataset.yaml', device='', entity=None, epochs=50, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp4', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='yolov5m6.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-06-01 06:02:33.439218: I tensorflow/stream_executor/platform/default/dso_

In [ ]:
%cd /content
time = datetime.today().strftime('%Y%m%d-%H%M')
shutil.copy2('yolov5/runs/train/exp/weights/last.pt', 'drive/MyDrive/作業/深度學習概論與應用/cdc/weights/OI_%s_last.pt'%(time))
shutil.copy2('yolov5/runs/train/exp/weights/best.pt', 'drive/MyDrive/作業/深度學習概論與應用/cdc/weights/OI_%s_best.pt'%(time))

/content


'drive/MyDrive/作業/深度學習概論與應用/cdc/weights/OI_20210601-0848_best.pt'

In [ ]:
%cd /content/yolov5
!python test.py --img 640 --conf-thres 0.00001 \
                --weights /content/yolov5/runs/train/exp/weights/last.pt \
                --data /content/open_images_v6/dataset.yaml

/content/yolov5
Namespace(augment=False, batch_size=32, conf_thres=1e-05, data='/content/open_images_v6/dataset.yaml', device='', exist_ok=False, img_size=640, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['/content/yolov5/runs/train/exp/weights/last.pt'])
YOLOv5 🚀 v5.0-130-gfdbe527 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 396 layers, 35445168 parameters, 0 gradients, 51.4 GFLOPS
val: Scanning '/content/train_cdc/labels/train.cache' images and labels... 2671 found, 0 missing, 2015 empty, 0 corrupted: 100% 2671/2671 [00:00<00:00, 27458298.98it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100% 84/84 [00:30<00:00,  2.78it/s]
                 all        2671        1148       0.826        0.81       0.904       0.754
              Toilet        2671          46       0.894   

In [ ]:
%cd /content/yolov5
!python detect.py --img 640 --conf-thres 0.00001 \
                  --source /content/test_cdc/test_images \
                  --weights /content/yolov5/runs/train/exp/weights/last.pt \
                  --save-txt --save-conf --nosave

/content/yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=1e-05, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=True, project='runs/detect', save_conf=True, save_crop=False, save_txt=True, source='/content/test_cdc/test_images', update=False, view_img=False, weights=['/content/yolov5/runs/train/exp/weights/last.pt'])
YOLOv5 🚀 v5.0-130-gfdbe527 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 396 layers, 35445168 parameters, 0 gradients, 51.4 GFLOPS
image 1/2248 /content/test_cdc/test_images/200801213.jpg: 512x640 21 Toilets, 11 Bathtubs, 183 Plastic bags, Done. (0.025s)
image 2/2248 /content/test_cdc/test_images/200801216.jpg: 512x640 112 Toilets, 142 Bathtubs, 85 Plastic bags, Done. (0.024s)
image 3/2248 /content/test_cdc/test_images/200801221.jpg: 512x640 54 Toilets, 195 Bathtubs, 68 Plastic bags, Done. (0.024s)
image 4/2248 /co

In [ ]:
%cd /content/yolov5
!python detect.py --img 448 \
                  --source /content/test_cdc/test_images \
                  --weights /content/yolov5/runs/train/exp/weights/last.pt

/content/yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=448, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/test_cdc/test_images', update=False, view_img=False, weights=['/content/yolov5/runs/train/exp/weights/last.pt'])
YOLOv5 🚀 v5.0-130-gfdbe527 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 396 layers, 35445168 parameters, 0 gradients, 51.4 GFLOPS
image 1/2248 /content/test_cdc/test_images/200801213.jpg: 384x448 Done. (0.020s)
image 2/2248 /content/test_cdc/test_images/200801216.jpg: 384x448 Done. (0.019s)
image 3/2248 /content/test_cdc/test_images/200801221.jpg: 384x448 1 Plastic bag, Done. (0.020s)
image 4/2248 /content/test_cdc/test_images/20080123.jpg: 384x448 1 Toilet, Done. (0.020s)
image 5/2248 /content/test_cdc/test_ima

In [ ]:
!cp -r /content/yolov5/runs/detect/exp2 /content/drive/MyDrive/作業/深度學習概論與應用

In [ ]:
%cd /content
!cp /content/drive/MyDrive/作業/深度學習概論與應用/submission-14.csv .

/content


In [ ]:
%cd /content
label_dir = 'yolov5/runs/detect/exp'
with open('submission.csv', 'w') as csv_file:
  csv_file.write('image_filename,label_id,x,y,w,h,confidence\n')

  _, _, filenames = next(os.walk(label_dir+'/labels'))
  image_ids = [os.path.splitext(filename)[0] for filename in filenames]

  for image_id in image_ids:
    with open(label_dir+'/labels/%s.txt'%(image_id)) as file:
      lines = file.readlines()
    for line in lines:
      data = line.split()
      im = Image.open('test_cdc/test_images/%s.jpg'%(image_id))
      w, h = im.size
      x_center = w * float(data[1])
      y_center = h * float(data[2])
      width = max(0, w * float(data[3]))
      height = max(0, h * float(data[4]))
      x_left = max(0, x_center - width/2)
      y_top = max(0, y_center - height/2)
      cdc_class = openImages_cdc_mapping[TRAIN_CLASSES[int(data[0])]]
      cdc_class_id = ALL_CLASSES.index(cdc_class) + 1

      result = '{}.jpg,{},{:.1f},{:.1f},{:.1f},{:.1f},{}\n'.format(
          image_id, cdc_class_id, x_left, y_top, width, height, data[5])
      csv_file.write(result)

/content


In [ ]:
train_cdc_ids = set(ALL_CLASSES.index(openImages_cdc_mapping[x])+1 for x in TRAIN_CLASSES)
with open('submission-14.csv', 'r') as other_csv_file, open('submission-removed.csv', 'w') as out_csv_file:
  rows = csv.DictReader(other_csv_file)
  writer = csv.DictWriter(out_csv_file, fieldnames=rows.fieldnames)
  for row in rows:
    if not int(row['label_id']) in train_cdc_ids:
      writer.writerow(row)

In [ ]:
!cat submission.csv submission-removed.csv > submission-20.csv

In [ ]:
import csv
with open('submission.csv', 'r') as in_file, open('submission-reduced.csv', 'w') as out_file:
  rows = csv.DictReader(in_file)
  writer = csv.DictWriter(out_file, fieldnames=rows.fieldnames)
  writer.writeheader()
  for row in rows:
    if float(row['confidence']) >= 0.01:
      writer.writerow(row)

In [ ]:
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 20 \
                --weights /content/yolov5/runs/train/exp/weights/last.pt \
                --data /content/open_images_v6/dataset.yaml

/content/yolov5
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-130-gfdbe527 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/open_images_v6/dataset.yaml', device='', entity=None, epochs=20, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp4', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='/content/yolov5/runs/train/exp/weights/last.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-06-01 09:16:26.514458: I tensorflow/st